In [13]:
from gladier.client import GladierClient as GladierBaseClient
from gladier_kanzus.flows.phils_beta_raf import flow_definition as flow_phil

class KanzusPhilsClient(GladierBaseClient):
    client_id = 'e6c75d97-532a-4c88-b031-8584a319fa3e'

    gladier_tools = [
        'gladier_kanzus.tools.CreatePhil',
        'gladier_kanzus.tools.DialsStills',
    ]
    flow_definition = flow_phil


phils_client = KanzusPhilsClient()


##hacking over container
from funcx.sdk.client import FuncXClient
fxc = FuncXClient()
from gladier_kanzus.tools.dials_stills import funcx_stills_process as stills_cont
cont_dir =  '/home/rvescovi/.funcx/containers/'
container_name = "dials_v1.simg"
dials_cont_id = fxc.register_container(location=cont_dir+'/'+container_name,container_type='singularity')
stills_cont_fxid = fxc.register_function(stills_cont, container_uuid=dials_cont_id)
##

##theta
# conf = {'local_endpoint': '8f2f2eab-90d2-45ba-a771-b96e6d530cad',
#         'queue_endpoint': '8f2f2eab-90d2-45ba-a771-b96e6d530cad',
#         }
##cooley
conf = {'local_endpoint': '83e95e2e-fd70-45ea-9467-5efe5d95ff11',
        'queue_endpoint': 'd26622fb-3bef-44df-8874-fcfdfbcc29fd',
        }

data_dir = '/eagle/APSDataAnalysis/SSX/S12/StTrpAB/E'
proc_dir = data_dir
proc_range = "{00001..00100}"

flow_input = {
    "input": {
        #Processing variables
        "proc_dir": proc_dir,
        "data_dir": data_dir,

        #Dials specific variables.
        "input_files": f"Emerge2_15_{proc_range}.cbf", 
        "input_range": proc_range[1:-1],
        "nproc": 20,
        "beamx": "-214.400",
        "beamy": "218.200",

        # funcX endpoints
        "funcx_local_ep": conf['local_endpoint'],
        "funcx_queue_ep": conf['queue_endpoint'],

        # container hack for stills
        "stills_cont_fxid": stills_cont_fxid
    }
}

phils_flow = phils_client.start_flow(flow_input=flow_input)
